In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime as dt

In [4]:
df = pd.read_csv("D:\DATA_Science\GitHub\BeforeGIT\sentimentMacbookAirM1.csv") # for Windows
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  76 non-null     object
dtypes: object(1)
memory usage: 736.0+ bytes


In [5]:
df

,message
0,มาคอนเฟิร์ม?อีกเสียงว่า Macbook?air? m1? ลื่นไ...
1,สอบถามได้มั้ยคะว่ากะจะซื้อ mac air m1 ไว้ทำงาน...
2,ผมยังใช้ M1 อยู่ถ้าไม่ได้ตัดต่อระดับเทพๆๆๆ ยัง...
3,ถ้าไม่นับเล่นเกมนะครับ นี่คือโน๊ตบุ๊คที่ดีที่ส...
4,จุดเด่นอีกอย่างคือ เสียงลำโพงมันดีมาก
...,...
71,Macbook air M1 ครับเพิ่งได้เครื่องมา ยังไม่ถึง...
72,"ถ้าเมื่อก่อนจะเชียร์ pro เสมอ, แต่ตอนนี้ผมว่าจ..."
73,ตามลักษณะ?การใช้งานเป็นหลัก สายตัดต่อวีดีโอ? ย...
74,ดูการทำงานแล้วก็งบด้วยนะครับถ้างบถึง pro ก็ดีค...
